# x402 Permit2 Demo — WETH Payments (Sepolia Testnets)

Test the **Permit2 payment flow** with **WETH** (Wrapped Ether) — a token that has **no EIP-3009 support**. This demonstrates the real power of Permit2: enabling x402 payments with *any* ERC-20 token, not just USDC.

## 🔄 Why WETH?

WETH is a standard ERC-20 without `transferWithAuthorization` (EIP-3009). Without Permit2, x402 simply **cannot** use WETH. This demo shows how Permit2 removes that limitation.

| Feature | EIP-3009 (USDC only) | Permit2 (any ERC-20) |
|---------|---------------------|---------------------|
| Approval target | USDC contract (native) | Canonical Permit2 contract |
| Signature type | `TransferWithAuthorization` | `PermitWitnessTransferFrom` |
| Token support | Only EIP-3009 tokens | **Any ERC-20 — including WETH** |
| One-time setup | None (built into USDC) | `WETH.approve(PERMIT2_ADDRESS, amount)` |

## ⚠️ Fee Challenge

The facilitator fee is always collected in **USDC** (not in the payment token). So even for a WETH payment, the *seller* still needs USDC allowance for fee collection. This creates an interesting UX consideration for non-USDC payments.

## 💰 Payment Flow (WETH via Permit2)

1. **One-time:** Buyer wraps ETH → WETH, then approves WETH to Permit2 contract
2. **Per payment:** Buyer signs a `PermitWitnessTransferFrom` for WETH
3. **Facilitator** verifies the Permit2 signature + checks seller's USDC fee allowance
4. **Facilitator** settles WETH via the x402 Permit2 Proxy contract
5. **Facilitator** collects the fee from seller **in USDC** (separate token!)

## 📋 Prerequisites

1. Start the local facilitator: `cd x402_facilitator && npm run dev`
2. Set env vars in `notebooks/.env`:
   - `TEST_WALLET_PRIVATE_KEY` — Buyer's private key (has ETH on Sepolia)
   - `NFT_WALLET_PUBLIC_KEY` — Seller's address (receives WETH)
   - `NFT_WALLET_PRIVATE_KEY` — Seller's private key (for USDC fee approval)

In [15]:
// Setup: Imports and Configuration
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
import { privateKeyToAccount } from "npm:viem@2/accounts";
import { createPublicClient, createWalletClient, http, formatUnits, encodeFunctionData, maxUint256 } from "npm:viem@2";
import { optimismSepolia, baseSepolia } from "npm:viem@2/chains";

// Load environment variables
const env = await load({ export: true });

const BUYER_PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const SELLER_ADDRESS = env.NFT_WALLET_PUBLIC_KEY as `0x${string}`;
const SELLER_PRIVATE_KEY = env.NFT_WALLET_PRIVATE_KEY;

// Create accounts
const buyerAccount = privateKeyToAccount(`0x${BUYER_PRIVATE_KEY}`);
const sellerAccount = privateKeyToAccount(`0x${SELLER_PRIVATE_KEY}`);

// Canonical Permit2 contract address (same on all EVM chains)
const PERMIT2_ADDRESS = "0x000000000022D473030F116dDEE9F6B43aC78BA3" as `0x${string}`;

console.log("🚀 x402 Permit2 Demo (Sepolia Testnets)");
console.log(`   Buyer:            ${buyerAccount.address}`);
console.log(`   Seller:           ${SELLER_ADDRESS}`);
console.log(`   Permit2 Contract: ${PERMIT2_ADDRESS}`);

🚀 x402 Permit2 Demo (Sepolia Testnets)
   Buyer:            0x553179556FC2A39e535D65b921e01fA995E79101
   Seller:           0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Permit2 Contract: 0x000000000022D473030F116dDEE9F6B43aC78BA3


## Network Configuration (Sepolia Only)

On all OP Stack chains (Optimism, Base), WETH is a **predeploy** at `0x4200000000000000000000000000000000000006`.

| Chain | Network ID | WETH Address | USDC Address (for fees) |
|-------|-----------|-------------|------------------------|
| Optimism Sepolia | `eip155:11155420` | `0x4200000000000000000000000000000000000006` | `0x5fd84259d66Cd46123540766Be93DFE6D43130D7` |
| Base Sepolia | `eip155:84532` | `0x4200000000000000000000000000000000000006` | `0x036CbD53842c5426634e7929541eC2318f3dCF7e` |

Toggle `USE_BASE` to switch between chains.

In [16]:
// ═══════════════════════════════════════════════════════════════
// ⚠️ NETWORK SELECTION — Sepolia testnets only
// ═══════════════════════════════════════════════════════════════

const USE_BASE = false; // true → Base Sepolia, false → Optimism Sepolia

// WETH predeploy address (same on all OP Stack L2s)
const WETH_ADDRESS = "0x4200000000000000000000000000000000000006" as `0x${string}`;
const WETH_DECIMALS = 18;

interface NetworkConfig {
    chain: typeof optimismSepolia;
    chainId: number;
    caip2Network: string;
    networkName: string;
    usdcAddress: `0x${string}`;
    explorerTxUrl: string;
}

const NETWORK_CONFIGS: Record<string, NetworkConfig> = {
    optimism: {
        chain: optimismSepolia,
        chainId: 11155420,
        caip2Network: "eip155:11155420",
        networkName: "Optimism Sepolia",
        usdcAddress: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
        explorerTxUrl: "https://sepolia-optimism.etherscan.io/tx/",
    },
    base: {
        chain: baseSepolia,
        chainId: 84532,
        caip2Network: "eip155:84532",
        networkName: "Base Sepolia",
        usdcAddress: "0x036CbD53842c5426634e7929541eC2318f3dCF7e",
        explorerTxUrl: "https://sepolia.basescan.org/tx/",
    },
};

const config = NETWORK_CONFIGS[USE_BASE ? "base" : "optimism"];
const chainConfig = config.chain;
const usdcAddress = config.usdcAddress;

// Payment amount: 0.00001 ETH (= 10_000_000_000_000 wei = 10^13)
const PAYMENT_AMOUNT = "10000000000000";

console.log(`🧪 Network: ${config.networkName}`);
console.log(`   Chain ID:  ${config.chainId}`);
console.log(`   CAIP-2:    ${config.caip2Network}`);
console.log(`   WETH:      ${WETH_ADDRESS} (18 decimals)`);
console.log(`   USDC:      ${usdcAddress} (for fees only)`);
console.log(`   Payment:   ${PAYMENT_AMOUNT} wei (${Number(PAYMENT_AMOUNT) / 1e18} ETH)`);

🧪 Network: Optimism Sepolia
   Chain ID:  11155420
   CAIP-2:    eip155:11155420
   WETH:      0x4200000000000000000000000000000000000006 (18 decimals)
   USDC:      0x5fd84259d66Cd46123540766Be93DFE6D43130D7 (for fees only)
   Payment:   10000000000000 wei (0.00001 ETH)


## Step 1: Query /supported

Check what the facilitator supports. Look for the **`permit2` extension** which advertises the canonical Permit2 contract address and the x402 Permit2 Proxy address.

In [17]:
// Facilitator endpoint (local dev server)
const FACILITATOR_URL = "http://localhost:8080";

const SUPPORTED_URL = `${FACILITATOR_URL}/supported`;
const VERIFY_URL = `${FACILITATOR_URL}/verify`;
const SETTLE_URL = `${FACILITATOR_URL}/settle`;

// Query /supported
const supportedResponse = await fetch(SUPPORTED_URL);
const supported = await supportedResponse.json();

console.log(`📡 GET ${SUPPORTED_URL} → ${supportedResponse.status}`);

// Show supported networks
console.log(`\n🌐 Supported Networks:`);
for (const kind of supported.kinds || []) {
    console.log(`   - ${kind.network} (scheme: ${kind.scheme}, x402v${kind.x402Version})`);
}

// Extract permit2 extension
const permit2Extension = supported.extensions?.find((e: any) => e.name === "permit2");
if (permit2Extension) {
    console.log(`\n🔑 Permit2 Extension:`);
    console.log(`   Permit2 Contract:  ${permit2Extension.permit2Address}`);
    console.log(`   x402 Proxy:        ${permit2Extension.proxyAddress}`);
    console.log(`   Methods:           ${permit2Extension.supportedMethods?.join(", ")}`);
    console.log(`   Description:       ${permit2Extension.description}`);
} else {
    console.log(`\n⚠️ No permit2 extension found — facilitator may not support Permit2`);
}

// Extract fee extension
const feeExtension = supported.extensions?.find((e: any) => e.name === "facilitator_fee");
let facilitatorAddress: string | null = null;
let feeAmount: string | null = null;

if (feeExtension) {
    facilitatorAddress = feeExtension.fee?.recipient;
    feeAmount = feeExtension.fee?.amount;
    console.log(`\n💸 Fee: ${feeExtension.fee?.description}`);
    console.log(`   Facilitator: ${facilitatorAddress}`);
} else {
    console.log(`\n💡 No fee extension — fees may be disabled`);
}

// Show signers
if (supported.signers?.["eip155:*"]) {
    console.log(`\n📝 Facilitator Signers: ${supported.signers["eip155:*"].join(", ")}`);
}

📡 GET http://localhost:8080/supported → 200

🌐 Supported Networks:
   - eip155:10 (scheme: exact, x402v2)
   - eip155:11155420 (scheme: exact, x402v2)
   - eip155:8453 (scheme: exact, x402v2)
   - eip155:84532 (scheme: exact, x402v2)

🔑 Permit2 Extension:
   Permit2 Contract:  0x000000000022D473030F116dDEE9F6B43aC78BA3
   x402 Proxy:        0x4020615294c913F045dc10f0a5cdEbd86c280001
   Methods:           eip3009, permit2
   Description:       Permit2 support for universal ERC-20 token payments. Clients can use either EIP-3009 (transferWithAuthorization) or Permit2 (PermitWitnessTransferFrom) to authorize payments. Permit2 requires a one-time token approval to the canonical Permit2 contract.

💸 Fee: 0.01 USDC per settlement
   Facilitator: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206

📝 Facilitator Signers: 0x0000000000000000000000000000000000000000


## Step 2: Check WETH Balances & Permit2 Allowance

For WETH payments via Permit2, the buyer needs:
1. Sufficient **WETH balance** (wrap ETH if needed)
2. Active **WETH approval to the Permit2 contract** (not the facilitator!)

The seller needs a **USDC** approval to the facilitator for fee collection — note the fee is in a *different token* than the payment!

In [18]:
// Minimal ERC-20 ABI (shared for WETH and USDC)
const ERC20_ABI = [
    {
        name: "balanceOf",
        type: "function",
        stateMutability: "view",
        inputs: [{ name: "account", type: "address" }],
        outputs: [{ name: "", type: "uint256" }],
    },
    {
        name: "allowance",
        type: "function",
        stateMutability: "view",
        inputs: [
            { name: "owner", type: "address" },
            { name: "spender", type: "address" },
        ],
        outputs: [{ name: "", type: "uint256" }],
    },
    {
        name: "approve",
        type: "function",
        stateMutability: "nonpayable",
        inputs: [
            { name: "spender", type: "address" },
            { name: "value", type: "uint256" },
        ],
        outputs: [{ name: "", type: "bool" }],
    },
] as const;

// WETH-specific: deposit() to wrap ETH
const WETH_ABI = [
    ...ERC20_ABI,
    {
        name: "deposit",
        type: "function",
        stateMutability: "payable",
        inputs: [],
        outputs: [],
    },
] as const;

// Create public client
const publicClient = createPublicClient({
    chain: chainConfig,
    transport: http(chainConfig.rpcUrls.default.http[0]),
});

// Helpers
async function getWethBalance(address: `0x${string}`): Promise<bigint> {
    return publicClient.readContract({
        address: WETH_ADDRESS,
        abi: WETH_ABI,
        functionName: "balanceOf",
        args: [address],
    });
}

async function getUsdcBalance(address: `0x${string}`): Promise<bigint> {
    return publicClient.readContract({
        address: usdcAddress,
        abi: ERC20_ABI,
        functionName: "balanceOf",
        args: [address],
    });
}

// Check WETH balances
const wethBalancesBefore = {
    buyer: await getWethBalance(buyerAccount.address),
    seller: await getWethBalance(SELLER_ADDRESS),
};

console.log(`💰 WETH Balances Before:`);
console.log(`   Buyer:  ${Number(wethBalancesBefore.buyer) / 1e18} WETH`);
console.log(`   Seller: ${Number(wethBalancesBefore.seller) / 1e18} WETH`);

// Check buyer's Permit2 allowance for WETH
const buyerPermit2Allowance = await publicClient.readContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "allowance",
    args: [buyerAccount.address, PERMIT2_ADDRESS],
});

console.log(`\n🔑 Buyer's WETH Permit2 Allowance:`);
if (buyerPermit2Allowance >= BigInt(PAYMENT_AMOUNT)) {
    console.log(`   ✅ ${Number(buyerPermit2Allowance) / 1e18} WETH approved to Permit2 (sufficient)`);
} else {
    console.log(`   ⚠️ ${Number(buyerPermit2Allowance) / 1e18} WETH — need at least ${Number(PAYMENT_AMOUNT) / 1e18}`);
    console.log(`   → Run Step 3a to wrap ETH and approve`);
}

// Check seller's USDC facilitator allowance (for fee — different token!)
if (facilitatorAddress) {
    const sellerFeeAllowance = await publicClient.readContract({
        address: usdcAddress,
        abi: ERC20_ABI,
        functionName: "allowance",
        args: [SELLER_ADDRESS, facilitatorAddress as `0x${string}`],
    });
    const feeAmountBigInt = feeAmount ? BigInt(feeAmount) : 10000n;
    console.log(`\n💸 Seller's USDC Fee Allowance (⚠️ different token than payment!):`);
    console.log(`   ${Number(sellerFeeAllowance) / 1e6} USDC approved for facilitator`);
    if (sellerFeeAllowance >= feeAmountBigInt) {
        console.log(`   ✅ Sufficient for fee collection`);
    } else {
        console.log(`   ⚠️ Insufficient — run Step 3b to approve`);
    }
}

💰 WETH Balances Before:
   Buyer:  0 WETH
   Seller: 0 WETH

🔑 Buyer's WETH Permit2 Allowance:
   ⚠️ 0 WETH — need at least 0.00001
   → Run Step 3a to wrap ETH and approve

💸 Seller's USDC Fee Allowance (⚠️ different token than payment!):
   0.98 USDC approved for facilitator
   ✅ Sufficient for fee collection


## Step 3a: Wrap ETH → WETH + Approve Permit2

Two setup transactions:
1. **Wrap ETH** — call `deposit()` on the WETH contract (sends ETH, receives WETH)
2. **Approve WETH to Permit2** — allow the canonical Permit2 contract to spend our WETH

> **Note:** The Permit2 contract address is the same on every EVM chain: `0x000000000022D473030F116dDEE9F6B43aC78BA3`

Skip if WETH balance and Permit2 allowance are already sufficient.

In [19]:
const buyerWalletClient = createWalletClient({
    account: buyerAccount,
    chain: chainConfig,
    transport: http(chainConfig.rpcUrls.default.http[0]),
});

// --- 1. Wrap ETH → WETH ---
const WRAP_AMOUNT = 100_000_000_000_000n; // 0.0001 ETH — enough for 10 payments

console.log(`📦 Wrapping ${Number(WRAP_AMOUNT) / 1e18} ETH → WETH...`);

const wrapTx = await buyerWalletClient.writeContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "deposit",
    value: WRAP_AMOUNT,
});

const wrapReceipt = await publicClient.waitForTransactionReceipt({ hash: wrapTx });
console.log(`   ✅ Wrapped in block ${wrapReceipt.blockNumber}`);
console.log(`   Explorer: ${config.explorerTxUrl}${wrapTx}`);

// --- 2. Approve WETH to Permit2 ---
const PERMIT2_APPROVAL_AMOUNT = 100_000_000_000_000n; // 0.0001 WETH

console.log(`\n📝 Approving ${Number(PERMIT2_APPROVAL_AMOUNT) / 1e18} WETH to Permit2...`);

const approveTx = await buyerWalletClient.writeContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "approve",
    args: [PERMIT2_ADDRESS, PERMIT2_APPROVAL_AMOUNT],
});

const approveReceipt = await publicClient.waitForTransactionReceipt({ hash: approveTx });
console.log(`   ✅ Approved in block ${approveReceipt.blockNumber}`);
console.log(`   Explorer: ${config.explorerTxUrl}${approveTx}`);

// Verify
const newWethBalance = await getWethBalance(buyerAccount.address);
const newWethAllowance = await publicClient.readContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "allowance",
    args: [buyerAccount.address, PERMIT2_ADDRESS],
});

console.log(`\n✅ Ready for WETH payments:`);
console.log(`   WETH Balance:      ${Number(newWethBalance) / 1e18} WETH`);
console.log(`   Permit2 Allowance: ${Number(newWethAllowance) / 1e18} WETH`);

📦 Wrapping 0.0001 ETH → WETH...
   ✅ Wrapped in block 39874205
   Explorer: https://sepolia-optimism.etherscan.io/tx/0xcb78ccca0a97ca9e17c5ab5d0eca5f1fa59f1eef11dd20964c1f5757cd20a80d

📝 Approving 0.0001 WETH to Permit2...
   ✅ Approved in block 39874207
   Explorer: https://sepolia-optimism.etherscan.io/tx/0x67b92abe5529bf9c9fd08eecd7598b5c7a55f17781f1e5ccf5421113bad334d4

✅ Ready for WETH payments:
   WETH Balance:      0.0001 WETH
   Permit2 Allowance: 0.0001 WETH


## Step 3b: Seller Approves USDC for Fee Collection

⚠️ **This is the fee challenge with non-USDC payments:** The buyer pays in WETH, but the facilitator fee is collected in **USDC** from the seller. The seller must hold USDC *and* approve the facilitator regardless of the payment token.

Skip if the seller already has sufficient USDC allowance.

In [ ]:
// Seller approves USDC for facilitator fee collection
// ⚠️ Note: Fee is in USDC even though payment is in WETH!
const feeAmountBigInt = feeAmount ? BigInt(feeAmount) : 10000n;
const APPROVAL_AMOUNT = 1_000_000n; // 1 USDC = covers 100 settlements at 0.01 USDC fee

const sellerWalletClient = createWalletClient({
    account: sellerAccount,
    chain: chainConfig,
    transport: http(chainConfig.rpcUrls.default.http[0]),
});

const facilitatorAddr = facilitatorAddress as `0x${string}`;

console.log(`📝 Seller approves USDC for fee collection...`);
console.log(`   ⚠️ Payment is in WETH, but fee is always in USDC!`);
console.log(`   Spender: ${facilitatorAddr} (facilitator)`);
console.log(`   Token:   ${usdcAddress} (USDC — not WETH!)`);
console.log(`   Amount:  ${Number(APPROVAL_AMOUNT) / 1e6} USDC`);

const feeApproveTx = await sellerWalletClient.writeContract({
    address: usdcAddress,
    abi: ERC20_ABI,
    functionName: "approve",
    args: [facilitatorAddr, APPROVAL_AMOUNT],
});

console.log(`   Tx Hash: ${feeApproveTx}`);
console.log(`   Waiting for confirmation...`);

const feeApproveReceipt = await publicClient.waitForTransactionReceipt({
    hash: feeApproveTx,
});

console.log(`   ✅ Approved in block ${feeApproveReceipt.blockNumber}`);
console.log(`   Explorer: ${config.explorerTxUrl}${feeApproveTx}`);

// Verify
const newSellerAllowance = await publicClient.readContract({
    address: usdcAddress,
    abi: ERC20_ABI,
    functionName: "allowance",
    args: [SELLER_ADDRESS, facilitatorAddr],
});
const remainingSettlements = feeAmountBigInt > 0n ? Number(newSellerAllowance / feeAmountBigInt) : Infinity;
console.log(`\n✅ Seller's USDC fee allowance: ${Number(newSellerAllowance) / 1e6} USDC (${remainingSettlements} settlements)`);

## Step 4: Create WETH Payment Payload

We set `extra.assetTransferMethod = "permit2"` — this is **mandatory** for WETH since it has no EIP-3009. The x402 client creates a `PermitWitnessTransferFrom` signature for WETH.

Key differences from USDC payments:
- `asset` → WETH address (`0x4200...0006`)
- `extra.name` → `"Wrapped Ether"` (WETH's ERC-20 name)
- `extra.version` → `"1"` (WETH contract version)
- Amount is in **wei** (18 decimals, not 6)

In [20]:
// Import x402 client packages
import type { PaymentRequirements, PaymentPayload, Network } from "npm:@x402/core@^2.3.1";
import { ExactEvmScheme } from "npm:@x402/evm@^2.3.1/exact/client";
import { x402Client } from "npm:@x402/fetch@^2.0.0";

// Create x402 client with ExactEvmScheme
const evmScheme = new ExactEvmScheme(buyerAccount);
const client = new x402Client();
client.register("eip155:*" as Network, evmScheme);

// Build PaymentRequirements for WETH via Permit2
const paymentRequirements: PaymentRequirements = {
    scheme: "exact",
    network: config.caip2Network,
    amount: PAYMENT_AMOUNT,
    asset: WETH_ADDRESS,         // ← WETH, not USDC!
    payTo: SELLER_ADDRESS,
    maxTimeoutSeconds: 3600,
    extra: {
        name: "Wrapped Ether",       // WETH's ERC-20 name
        version: "1",                // WETH contract version
        assetTransferMethod: "permit2",  // mandatory — WETH has NO EIP-3009!
    }
};

// Build mock 402 response (normally comes from server)
const paymentRequired = {
    x402Version: 2,
    accepts: [paymentRequirements],
    resource: {
        url: "https://example.com/resource",
        description: "Test payment via Permit2 with WETH",
        mimeType: "application/json"
    },
    extensions: {}
};

// Create payment payload — buyer signs PermitWitnessTransferFrom for WETH
const paymentPayload: PaymentPayload = await client.createPaymentPayload(paymentRequired as any);

// Inspect the payload structure
const p2auth = (paymentPayload as any).payload?.permit2Authorization;
const eip3009auth = (paymentPayload as any).payload?.authorization;

if (p2auth) {
    console.log("✅ WETH Permit2 Payment Payload created!");
    console.log(`   Transfer Method: Permit2 (PermitWitnessTransferFrom)`);
    console.log(`   Token:           ${p2auth.permitted?.token} (WETH)`);
    console.log(`   Amount:          ${p2auth.permitted?.amount} wei (${Number(p2auth.permitted?.amount || 0) / 1e18} ETH)`);
    console.log(`   From:            ${p2auth.from}`);
    console.log(`   To (witness):    ${p2auth.witness?.to}`);
    console.log(`   Nonce:           ${p2auth.nonce}`);
    console.log(`   Deadline:        ${p2auth.deadline}`);
    console.log(`   ValidAfter:      ${p2auth.witness?.validAfter}`);
} else if (eip3009auth) {
    console.log("❌ Got EIP-3009 payload — but WETH doesn't support EIP-3009!");
    console.log("   This should not happen with assetTransferMethod: 'permit2'");
} else {
    console.log("❌ Unknown payload structure:");
    console.log(JSON.stringify(paymentPayload, null, 2));
}

✅ WETH Permit2 Payment Payload created!
   Transfer Method: Permit2 (PermitWitnessTransferFrom)
   Token:           0x4200000000000000000000000000000000000006 (WETH)
   Amount:          10000000000000 wei (0.00001 ETH)
   From:            0x553179556FC2A39e535D65b921e01fA995E79101
   To (witness):    0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Nonce:           89362397400758616246228114686223067545195438203708742048337237856170446855735
   Deadline:        1771554579
   ValidAfter:      1771550379


## Step 5: Verify Payment

Send the WETH payment to `/verify`. The facilitator will:
1. Detect the `permit2Authorization` field in the payload
2. Verify the Permit2 signature (PermitWitnessTransferFrom) for WETH
3. Check buyer's WETH balance
4. Check seller's **USDC** allowance for fee collection (⚠️ different token!)

In [21]:
// Build verify request
const verifyRequest = {
    paymentPayload: paymentPayload,
    paymentRequirements: paymentRequirements
};

console.log("🔍 Verifying Permit2 payment...");

const verifyResponse = await fetch(VERIFY_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const verifyResult = await verifyResponse.json();

console.log(`\n📦 Verify Response (Status ${verifyResponse.status}):`);
console.log(JSON.stringify(verifyResult, null, 2));

if (verifyResult.isValid) {
    console.log(`\n✅ Permit2 payment is VALID!`);
    console.log(`   Payer: ${verifyResult.payer}`);
    if (verifyResult.feeRequired !== undefined) {
        console.log(`   Fee Required: ${verifyResult.feeRequired}`);
    }
    if (verifyResult.recipient) {
        console.log(`   Recipient (Seller): ${verifyResult.recipient}`);
    }
} else {
    console.log(`\n❌ Payment is INVALID!`);
    console.log(`   Reason: ${verifyResult.invalidReason}`);
    if (verifyResult.invalidReason === "insufficient_fee_allowance") {
        console.log(`\n💡 Fix: Run Step 3b to approve seller's fee allowance`);
    }
}

🔍 Verifying Permit2 payment...

📦 Verify Response (Status 200):
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

✅ Permit2 payment is VALID!
   Payer: 0x553179556FC2A39e535D65b921e01fA995E79101


## Step 6: Settle Payment

The facilitator routes the WETH Permit2 payload through the **x402 Permit2 Proxy** contract. The WETH transfer and the USDC fee collection happen as **separate transactions**.

Fund flow:
- `WETH`: Buyer → Seller (full amount via Permit2 Proxy)
- `USDC`: Seller → Facilitator (fee via `transferFrom`)

In [ ]:
// ═══════════════════════════════════════════════════════════════
// Snapshot balances BEFORE settlement (after setup steps 3a/3b)
// ═══════════════════════════════════════════════════════════════
const wethBeforeSettle = {
    buyer: await getWethBalance(buyerAccount.address),
    seller: await getWethBalance(SELLER_ADDRESS),
};
const usdcBeforeSettle = {
    seller: await getUsdcBalance(SELLER_ADDRESS),
    facilitator: facilitatorAddress ? await getUsdcBalance(facilitatorAddress as `0x${string}`) : 0n,
};
const permit2AllowanceBefore = await publicClient.readContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "allowance",
    args: [buyerAccount.address, PERMIT2_ADDRESS],
});

console.log(`📸 Pre-Settlement Snapshot:`);
console.log(`   Buyer WETH:       ${Number(wethBeforeSettle.buyer) / 1e18}`);
console.log(`   Seller WETH:      ${Number(wethBeforeSettle.seller) / 1e18}`);
console.log(`   Seller USDC:      ${Number(usdcBeforeSettle.seller) / 1e6}`);
console.log(`   Facilitator USDC: ${Number(usdcBeforeSettle.facilitator) / 1e6}`);
console.log(`   Permit2 Allowance: ${Number(permit2AllowanceBefore) / 1e18} WETH`);

// Fee amount (from /supported or default)
const feeAmountBigInt = feeAmount ? BigInt(feeAmount) : 10000n;

console.log(`\n💸 Settling WETH Permit2 payment...`);
console.log(`   Network:  ${config.caip2Network}`);
console.log(`   Method:   Permit2 (PermitWitnessTransferFrom)`);
console.log(`   WETH:     Buyer → Seller: ${Number(PAYMENT_AMOUNT) / 1e18} ETH`);
console.log(`   USDC Fee: Seller → Facilitator: ${Number(feeAmountBigInt) / 1e6} USDC`);

const settleResponse = await fetch(SETTLE_URL, {
    method: "POST",
    headers: { "Content-Type": "application/json" },
    body: JSON.stringify(verifyRequest)
});

const settleResult = await settleResponse.json();

console.log(`\n📦 Settle Response (Status ${settleResponse.status}):`);
console.log(JSON.stringify(settleResult, null, 2));

if (settleResult.success) {
    console.log(`\n🎉 WETH settlement successful!`);
    console.log(`   Transaction: ${settleResult.transaction}`);
    console.log(`   Explorer:    ${config.explorerTxUrl}${settleResult.transaction}`);
    if (settleResult.network) {
        console.log(`   Network:     ${settleResult.network}`);
    }
    // Fee details
    if (settleResult.fee) {
        console.log(`\n💰 USDC Fee Collection:`);
        console.log(`   Collected: ${settleResult.fee.collected}`);
        if (settleResult.fee.txHash) {
            console.log(`   Fee Tx:    ${settleResult.fee.txHash}`);
            console.log(`   Explorer:  ${config.explorerTxUrl}${settleResult.fee.txHash}`);
        }
        if (settleResult.fee.error) {
            console.log(`   ⚠️ Fee Error: ${settleResult.fee.error}`);
        }
    }
    // Extensions
    if (settleResult.extensions?.facilitatorFees) {
        const fees = settleResult.extensions.facilitatorFees.info;
        console.log(`\n📊 Fee Receipt (x402 #1016):`);
        console.log(`   Paid:  ${fees.facilitatorFeePaid} (${Number(fees.facilitatorFeePaid) / 1e6} USDC)`);
        console.log(`   Model: ${fees.model}`);
        console.log(`   Asset: ${fees.asset}`);
    }
} else {
    console.log(`\n❌ Settlement failed`);
    console.log(`   Reason: ${settleResult.errorReason || "unknown"}`);
}

💸 Settling WETH Permit2 payment...
   Network:  eip155:11155420
   Method:   Permit2 (PermitWitnessTransferFrom)
   WETH:     Buyer → Seller: 0.00001 ETH
   USDC Fee: Seller → Facilitator: 0.01 USDC

📦 Settle Response (Status 200):
{
  "success": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "0x56796d86e771e5719c8e22f6bda055ebd553d6bc7a068d24dbed0e7cc0a59af0",
  "network": "eip155:11155420",
  "fee": {
    "collected": true,
    "txHash": "0xbf8340e43e1b90c7095b7ce78e84e234d94917edafe791747a699cab268de99f"
  },
  "extensions": {
    "facilitatorFees": {
      "info": {
        "version": "1",
        "facilitatorFeePaid": "10000",
        "asset": "eip155:11155420/erc20:0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
        "model": "flat"
      }
    }
  }
}

🎉 WETH settlement successful!
   Transaction: 0x56796d86e771e5719c8e22f6bda055ebd553d6bc7a068d24dbed0e7cc0a59af0
   Explorer:    https://sepolia-optimism.etherscan.io/tx/0x56796d86e771e5719c8e22f6b

## Step 7: Post-Settlement Verification

Check **two tokens** after settlement:
- **WETH**: Buyer paid → Seller received (full amount, no fee deducted from WETH)
- **USDC**: Facilitator collected fee from Seller

This is the key difference from USDC payments: the payment and fee involve *different tokens*.

In [ ]:
// Check WETH balances AFTER settlement (compared to pre-settle snapshot)
const wethAfterSettle = {
    buyer: await getWethBalance(buyerAccount.address),
    seller: await getWethBalance(SELLER_ADDRESS),
};

const wethBuyerDelta = wethBeforeSettle.buyer - wethAfterSettle.buyer;
const wethSellerDelta = wethAfterSettle.seller - wethBeforeSettle.seller;

console.log(`💰 WETH Balance Changes (pre-settle → post-settle):`);
console.log(`   Buyer:  ${Number(wethBeforeSettle.buyer) / 1e18} → ${Number(wethAfterSettle.buyer) / 1e18} WETH (spent ${Number(wethBuyerDelta) / 1e18})`);
console.log(`   Seller: ${Number(wethBeforeSettle.seller) / 1e18} → ${Number(wethAfterSettle.seller) / 1e18} WETH (gained ${Number(wethSellerDelta) / 1e18})`);

// Check Permit2 WETH allowance after (decreases with each payment)
const permit2AllowanceAfter = await publicClient.readContract({
    address: WETH_ADDRESS,
    abi: WETH_ABI,
    functionName: "allowance",
    args: [buyerAccount.address, PERMIT2_ADDRESS],
});
const allowanceDelta = permit2AllowanceBefore - permit2AllowanceAfter;
console.log(`\n🔑 Permit2 Allowance: ${Number(permit2AllowanceBefore) / 1e18} → ${Number(permit2AllowanceAfter) / 1e18} WETH (consumed ${Number(allowanceDelta) / 1e18})`);

// Diagnostics: did the Permit2 transfer actually execute?
if (allowanceDelta === 0n && wethBuyerDelta === 0n) {
    console.log(`\n⚠️ DIAGNOSTIC: Permit2 allowance unchanged + buyer balance unchanged`);
    console.log(`   → The Permit2 transfer did NOT execute on-chain!`);
    console.log(`   Possible causes:`);
    console.log(`   1. Nonce already used (re-ran settle with same payload)`);
    console.log(`   2. Permit2 proxy silently failed (check internal txs on Etherscan)`);
    console.log(`   3. Signature mismatch (re-create payload in Step 4)`);
    console.log(`   → Check: ${config.explorerTxUrl}${settleResult.transaction} → "Internal Txns" tab`);
}

// Verify WETH transfer
console.log(`\n✅ WETH Verification:`);
if (wethBuyerDelta === BigInt(PAYMENT_AMOUNT)) {
    console.log(`   ✅ Buyer paid: ${Number(wethBuyerDelta) / 1e18} WETH`);
} else {
    console.log(`   ❌ Buyer delta: expected ${Number(PAYMENT_AMOUNT) / 1e18}, got ${Number(wethBuyerDelta) / 1e18}`);
}

// With WETH, the full payment goes to seller (fee is in USDC, not deducted from WETH)
if (wethSellerDelta === BigInt(PAYMENT_AMOUNT)) {
    console.log(`   ✅ Seller received full amount: ${Number(wethSellerDelta) / 1e18} WETH (no fee deducted — fee is in USDC)`);
} else {
    console.log(`   ❓ Seller received: ${Number(wethSellerDelta) / 1e18} WETH (expected ${Number(PAYMENT_AMOUNT) / 1e18})`);
}

// Check USDC fee changes
if (facilitatorAddress) {
    const usdcAfterSettle = {
        seller: await getUsdcBalance(SELLER_ADDRESS),
        facilitator: await getUsdcBalance(facilitatorAddress as `0x${string}`),
    };
    const usdcSellerDelta = usdcBeforeSettle.seller - usdcAfterSettle.seller;
    const usdcFacilitatorDelta = usdcAfterSettle.facilitator - usdcBeforeSettle.facilitator;
    console.log(`\n💸 USDC Fee Verification:`);
    console.log(`   Seller:      ${Number(usdcBeforeSettle.seller) / 1e6} → ${Number(usdcAfterSettle.seller) / 1e6} USDC (paid ${Number(usdcSellerDelta) / 1e6})`);
    console.log(`   Facilitator: ${Number(usdcBeforeSettle.facilitator) / 1e6} → ${Number(usdcAfterSettle.facilitator) / 1e6} USDC (received ${Number(usdcFacilitatorDelta) / 1e6})`);
}

console.log(`\n🎯 Key Takeaway:`);
console.log(`   WETH has NO EIP-3009 support → impossible via x402 without Permit2.`);
console.log(`   Permit2 enables x402 payments with ANY ERC-20 token.`);
console.log(`   But: fees are still in USDC — seller must hold & approve USDC regardless.`);

💰 WETH Balance Changes:
   Buyer:  0 → 0.0001 WETH (spent -0.0001)
   Seller: 0 → 0 WETH (gained 0)

🔑 Buyer's WETH Permit2 Allowance: 0.0001 WETH

✅ WETH Verification:
   ❌ Buyer delta: expected 0.00001, got -0.0001
   ❓ Seller received: 0 WETH (expected 0.00001)

💸 USDC Fee Verification:
   Facilitator USDC: 0.06 USDC
   Seller USDC:      1.471 USDC

🎯 Key Takeaway:
   WETH has NO EIP-3009 support → impossible via x402 without Permit2.
   Permit2 enables x402 payments with ANY ERC-20 token.
   But: fees are still in USDC — seller must hold & approve USDC regardless.


## Summary

This notebook demonstrates **WETH payments via Permit2** through our x402 facilitator — something **impossible** with EIP-3009 alone.

| Step | What happens |
|------|-------------|
| 1. `/supported` | Returns `permit2` extension with contract addresses |
| 2. Balance check | WETH balances + Permit2 allowance + seller's USDC fee allowance |
| 3a. Buyer setup | Wrap ETH → WETH, then `WETH.approve(PERMIT2_ADDRESS, amount)` |
| 3b. Seller setup | `USDC.approve(facilitator, amount)` for fees (⚠️ different token!) |
| 4. Payment creation | `asset: WETH`, `assetTransferMethod: "permit2"` → `PermitWitnessTransferFrom` |
| 5. `/verify` | Validates Permit2 signature for WETH + USDC fee allowance |
| 6. `/settle` | Settles WETH via Permit2 Proxy, collects USDC fee separately |
| 7. Verification | WETH: buyer → seller (full), USDC: seller → facilitator (fee) |

### The Fee Challenge

With USDC payments, fees are deducted from the same token. With WETH payments, the fee is in a **different token** (USDC):
- Seller receives **full WETH amount** (no deduction)
- Seller pays fee **in USDC** separately
- Seller must hold USDC *and* have approved the facilitator — even for WETH payments

This is an inherent limitation when the payment token ≠ fee token.

### When to Use Permit2

- **EIP-3009** is simpler for USDC payments (no extra approval step)
- **Permit2** is required when:
  - The token has no EIP-3009 support (WETH, DAI, any standard ERC-20)
  - You want a universal flow across all ERC-20 tokens
  - The buyer already has a Permit2 approval (common for DeFi users)